<a href="https://colab.research.google.com/github/inderpreetsingh01/Speech/blob/main/Speaker_Identification_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers[sentencepiece] librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 33.6 MB/s 
     |████████████████████████████████| 69 kB 11.2 MB/s 
     |████████████████████████████████| 4.7 MB 59.0 MB/s 
     |████████████████████████████████| 101 kB 15.7 MB/s 
     |████████████████████████████████| 212 kB 71.3 MB/s 
     |████████████████████████████████| 115 kB 75.3 MB/s 
     |████████████████████████████████| 141 kB 67.3 MB/s 
     |████████████████████████████████| 596 kB 65.4 MB/s 
     |████████████████████████████████| 127 kB 74.2 MB/s 
     |████████████████████████████████| 6.6 MB 63.9 MB/s 
     |████████████████████████████████| 1.3 MB 50.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:

In [2]:
import torch
# import librosa
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor, DataCollatorWithPadding

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

## Data Loading

In [4]:
dataset_streamed = load_dataset("librispeech_asr", split="train.clean.100", streaming=True)
# next(iter(dataset_streamed))
dataset = list(dataset_streamed.take(500-37))
dataset[0]

No config specified, defaulting to: librispeech_asr/all


{'audio': {'array': array([ 7.01904297e-04,  7.32421875e-04,  7.32421875e-04, ...,
         -2.74658203e-04, -1.83105469e-04, -3.05175781e-05]),
  'path': '374-180298-0000.flac',
  'sampling_rate': 16000},
 'chapter_id': 180298,
 'file': '374-180298-0000.flac',
 'id': '374-180298-0000',
 'speaker_id': 374,
 'text': 'CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE BEGINNING OF THIS LIAISON IN A FEW LINES BUT I WANTED YOU TO SEE EVERY STEP BY WHICH WE CAME I TO AGREE TO WHATEVER MARGUERITE WISHED'}

In [5]:
speaker_id = []
for data in dataset:
  speaker_id.append(data['speaker_id'])
speaker_id_df = pd.DataFrame(speaker_id)
speaker_id_df.value_counts()

3240    127
7800    115
374     113
2514    108
dtype: int64

In [6]:
speaker_id_df[0].unique()

array([ 374, 7800, 2514, 3240])

In [7]:
train_dataset = []
test_dataset = []
speaker_id2label = {}

for i, speaker_id in enumerate(speaker_id_df[0].unique()):
  speaker_id2label[speaker_id] = i+1
  j=0
  for data in dataset:
    if (data['speaker_id'] == speaker_id):
      if (j<100):
        train_dataset.append({'label':i+1, 'audio':data['audio']['array']})
      else:
        test_dataset.append({'label':i+1, 'audio':data['audio']['array']})
      j+=1

In [8]:
speaker_id2label

{374: 1, 2514: 3, 3240: 4, 7800: 2}

In [9]:
type(test_dataset[0]['audio'])

numpy.ndarray

In [10]:
test_dataset[0]['audio'].shape

(256560,)

In [11]:
print(f"""
Number of training samples: {len(train_dataset)}
Number of test samples: {len(test_dataset)}
""")


Number of training samples: 400
Number of test samples: 63



In [12]:
train_df = pd.DataFrame(train_dataset)
train_df.head()

,label,audio
0,1,"[0.000701904296875, 0.000732421875, 0.00073242..."
1,1,"[-9.1552734375e-05, -0.000152587890625, -0.000..."
2,1,"[-0.000244140625, -0.000244140625, -0.00018310..."
3,1,"[-0.000244140625, -0.000396728515625, -0.00057..."
4,1,"[0.000274658203125, 0.00030517578125, 0.000213..."


In [13]:
test_df = pd.DataFrame(test_dataset)
test_df.head()

,label,audio
0,1,"[0.0, -3.0517578125e-05, 0.0, -6.103515625e-05..."
1,1,"[-0.000457763671875, -0.00048828125, -0.000457..."
2,1,"[0.000579833984375, 0.0003662109375, 0.0001525..."
3,1,"[-0.000701904296875, -0.00054931640625, -0.000..."
4,1,"[9.1552734375e-05, 3.0517578125e-05, 9.1552734..."


In [ ]:
for i in list(train_df["audio"]):
  print(type(i))

In [15]:
checkpoint = "superb/wav2vec2-base-superb-sid"
model = Wav2Vec2ForSequenceClassification.from_pretrained(checkpoint)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(checkpoint)

# compute attention masks and normalize the waveform if needed
# inputs = feature_extractor(list(train_df["audio"]), sampling_rate=16000, padding=True, return_tensors="pt")

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [16]:
def tokenize_function(example):
    return feature_extractor(list(example["audio"]), sampling_rate=16000, padding=True, return_tensors="pt")

data_collator = DataCollatorWithPadding(tokenizer=tokenize_function)

In [17]:
from torch.utils.data import Dataset
class EasyDataset(Dataset):

    def __init__(self,df,
                 feature_extractor, 
              ):
        self.df = df
        self.feature_extractor = feature_extractor
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        audio = self.df["audio"][idx]
        label = self.df["label"][idx]

        """ When Transformers are used for V backbone"""
        audio_features = self.feature_extractor(audio, sampling_rate=16000, padding='max_length', max_length=300000, return_tensors="pt")
        audio_features = {k:v.reshape((300000,)) for k,v in audio_features.items()}
        encoding = {}
        encoding['input_values'] = audio_features['input_values']
        encoding['attention_mask'] = audio_features['attention_mask']
        encoding['labels'] = torch.from_numpy(np.array(label)).reshape((1,))
        return encoding

train_dataset = EasyDataset(
                           df=train_df,
                           feature_extractor = feature_extractor
                           )

test_dataset = EasyDataset(
                           df=test_df,
                           feature_extractor = feature_extractor
                           )

In [18]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset, shuffle=True, batch_size=8 
)
test_dataloader = DataLoader(
    test_dataset, batch_size=8
)

In [19]:
for data in train_dataloader:
  print(data)
  print(data['input_values'].shape)
  print(data['attention_mask'].shape)
  print(data['labels'].shape)
  break

{'input_values': tensor([[ 5.0049e-03, -5.4932e-04, -3.0518e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.0986e-03,  0.0000e+00, -5.1880e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -3.0518e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-2.1362e-04,  1.5259e-04, -6.7139e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 7.9346e-04, -1.9531e-03,  7.0190e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32), 'labels': tensor([[3],
        [1],
        [4],
        [2],
        [2],
        [4],
      

In [20]:
checkpoint = "superb/wav2vec2-base-superb-sid"
model = Wav2Vec2ForSequenceClassification.from_pretrained(checkpoint)
# feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(checkpoint)

# compute attention masks and normalize the waveform if needed
# inputs = feature_extractor(dataset[:2]["speech"], sampling_rate=16000, padding=True, return_tensors="pt")

# logits = model(**inputs).logits
# predicted_ids = torch.argmax(logits, dim=-1)
# labels = [model.config.id2label[_id] for _id in predicted_ids.tolist()]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [21]:
model.to(device)

Wav2Vec2ForSequenceClassification(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), strid

In [22]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [23]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

150


In [24]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    print(f'Epoch {epoch}')
    for iter, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        print(f'loss: {loss}, iter: {iter+1}')
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 0
loss: 28.25666618347168, iter: 1
loss: 13.736190795898438, iter: 2
loss: 6.048591613769531, iter: 3
loss: 0.14776165783405304, iter: 4
loss: 1.4439949989318848, iter: 5
loss: 4.573954105377197, iter: 6
loss: 2.345080614089966, iter: 7
loss: 0.00017403504170943052, iter: 8
loss: 1.2901263236999512, iter: 9
loss: 0.0006846595206297934, iter: 10
loss: 1.7191940546035767, iter: 11
loss: 0.0, iter: 12
loss: 0.04335980862379074, iter: 13
loss: 2.9354966954997508e-06, iter: 14
loss: 7.122574970708229e-06, iter: 15
loss: 1.5199092331386055e-06, iter: 16
loss: 2.011641072385828e-06, iter: 17
loss: 0.0, iter: 18
loss: 0.004065264016389847, iter: 19
loss: 0.00038298399886116385, iter: 20
loss: 0.029704883694648743, iter: 21
loss: 0.00027065465110354125, iter: 22
loss: 0.00017751063569448888, iter: 23
loss: 0.00045197439612820745, iter: 24
loss: 7.599568903060572e-07, iter: 25
loss: 8.733435970498249e-05, iter: 26
loss: 2.9802320611338473e-08, iter: 27
loss: 3.009998408742831e-06, iter: 28

In [25]:
# import evaluate

# metric = evaluate.load("librispeech_asr")
predictions = []
labels = []

model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
      
    labels.append(batch['labels'].reshape(-1,).cpu().numpy())
    logits = outputs.logits
    predictions.append(torch.argmax(logits, dim=-1).reshape(-1,).cpu().numpy())
    # metric.add_batch(predictions=predictions, references=batch["labels"])
# metric.compute()

labels = np.concatenate(labels, axis=0)
predictions = np.concatenate(predictions, axis=0)
accuracy = (labels == predictions).sum()/len(predictions)
print(f"Accuracy: {accuracy*100}")

Accuracy: 100.0


In [105]:
def map_to_array(example):
    speech, _ = librosa.load(example["file"], sr=16000, mono=True)
    example["speech"] = speech
    return example

# load a demo dataset and read audio files
dataset = load_dataset("anton-l/superb_demo", "si", split="test")
dataset = dataset.map(map_to_array)

model = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-base-superb-sid")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-sid")

# compute attention masks and normalize the waveform if needed
inputs = feature_extractor(dataset[:2]["speech"], sampling_rate=16000, padding=True, return_tensors="pt")

logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)
labels = [model.config.id2label[_id] for _id in predicted_ids.tolist()]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [106]:
labels

['id10003', 'id10004']

In [107]:
dataset


Dataset({
    features: ['file', 'audio', 'label', 'speech'],
    num_rows: 6
})

In [114]:
len(dataset[:2]["speech"][1])

284161